The purpose of this notebook is to demonstrate the prototype gene mapper library put together to generalize the processes of

- Mapping gene symbols to gene identifiers
- Mapping gene identifiers across authorities (NCBI, ENSEMBL, etc.)
- Mapping genes in one species to ortholog genes in another species

for MapMyCells.

In addition to the `mmc_gene_mapper` codebase and its dependencies, this notebook depends on the `abc_atlas_access` tool, though only for downloading metadata to simulate mapping 10s of thousands of genes at once (as we would with real user data).

## Setup

In [1]:
import h5py
import json
import pathlib
import subprocess
import sqlite3
import time

import abc_atlas_access.abc_atlas_cache.abc_project_cache as abc_cache_library

import mmc_gene_mapper
import mmc_gene_mapper.utils.file_utils as file_utils
import mmc_gene_mapper.ensembl_download.scraper as ensembl_scraper
import mmc_gene_mapper.mapper.mapper as mapper

/Users/scott.daniel/miniconda3/envs/gene_mapper/lib/python3.12/site-packages/bkbit/data_translators/genome_annotation_translator.py:67: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
abc_cache = abc_cache_library.AbcProjectCache.from_cache_dir('../data/abc_cache')

/Users/scott.daniel/miniconda3/envs/gene_mapper/lib/python3.12/site-packages/abc_atlas_access/abc_atlas_cache/cloud_cache.py:630: OutdatedManifestWarning: You are loading releases/20250331/manifest.json. A more up to date version of the dataset -- releases/20250531/manifest.json -- exists online. To see the changes between the two versions of the dataset, run
type.compare_manifests('releases/20250331/manifest.json', 'releases/20250531/manifest.json')
To load another version of the dataset, run
type.load_manifest('releases/20250531/manifest.json')
  warnings.warn(msg, OutdatedManifestWarning)


In [3]:
bican_files = [
    {'url': 'https://ftp.ensembl.org/pub/release-101/gff3/homo_sapiens/Homo_sapiens.GRCh38.101.gff3.gz',
     'assembly_id': 'GCF_000001405.39'},
    {'url': 'https://ftp.ensembl.org/pub/release-98/gff3/mus_musculus/Mus_musculus.GRCm38.98.gff3.gz',
     'assembly_id': 'GCF_000001635.26'}
]

In [ ]:
t0 = time.time()
ensembl_files_spec = ensembl_scraper.scrape_ensembl(
    default_ensembl_version=114,
    dst_dir='../data/ensembl_download',
    failure_log_path='../data/ensembl_download_failure.txt',
    specific_files=bican_files,
    tmp_dir='../data/scratch'
)
dur = (time.time()-t0)/60.0
print(f'scraping ENSEMBL took {dur:.2e} minutes')

In [5]:
data_dir = pathlib.Path(
    mmc_gene_mapper.__file__).parent.parent.parent / "data"
assert data_dir.is_dir()

In [6]:
db_path = data_dir/"gene_mapper_example.db"
assert db_path.parent.is_dir()

There are some files in
```
/allen/scratch/aibstemp/danielsf/gene_mapper_data
```
that are used to ingest data beyond what is available in the NCBI FTP server. Please copy those files to `../data/db_creation_data` in this respository (the cell below will fail if any of the files are not present)

In [ ]:
data_file_spec=[
    {"type": "bkbit",
     "path": "../data/db_creation_data/mouse_ENSEMBL-10090-98.jsonld"},
    {"type": "bkbit",
     "path": "../data/db_creation_data/human_ENSEMBL-9606-101.jsonld"},
    {"type": "hmba_orthologs",
     "path": "../data/db_creation_data/all_gene_ids.csv",
     "name": "HMBA",
     "baseline_species": "human"}
]

error_msg = ""
for spec in data_file_spec:
    pth = pathlib.Path(spec['path'])
    if not pth.is_file():
        error_msg += f"{pth} is not a file\n"
if len(error_msg) > 0:
    raise RuntimeError(error_msg)

Below we instantiate the class that is used to actually do the mapping. The first time you run the cell, it will take ~ 30 minutes as it downloads ~ 4 GB of data from NCBI and ingests that data, alongside the local data from the cell above, to create an ~ 8 GB sqlite database that backs the mapping process.

As long as you leave `clobber=False` below, you will not have to recreate that database on subsequent uses.

**Note:** running this cell will also produce some warnings about genes that could not be ingested into the ortholog tables. These are due to genes in the ortholog table that have been deprecated by NCBI since the ortholog table (especially our by-hand HMBA ortholog table) was generated, meaning the gene mapper cannot connect them to species. We will have to think about how to handle versioning mismatches of this sort.

In [ ]:
ensembl_files_spec = [{'type': 'bkbit', 'path': p} for p in ensembl_files_spec]

In [7]:
ensembl_dir = data_dir / 'ensembl_download'
assert ensembl_dir.is_dir()
ensembl_files_spec = [
    {'type': 'bkbit', 'path': p}
    for p in ensembl_dir.rglob('**/*jsonld')
]

In [8]:
ensembl_files_spec[0]

{'type': 'bkbit',
 'path': PosixPath('/Users/scott.daniel/KnowledgeEngineering/mmc_gene_mapper/data/ensembl_download/monodelphis_domestica.jsonld')}

In [9]:
gene_mapper = mapper.MMCGeneMapper(
    db_path=db_path,
    local_dir=data_dir,
    data_file_spec=ensembl_files_spec,
    clobber=True,
    force_download=False,
    suppress_download_stdout=True
)

=======CREATING DB FILE=======
=======CREATING TABLES=======
=======INGESTING GENE INFO=======
    chunk 0.00e+00, 5.00e+06
    chunk 5.00e+06, 1.00e+07
    chunk 1.00e+07, 1.50e+07
    chunk 1.50e+07, 2.00e+07
    chunk 2.00e+07, 2.50e+07
    chunk 2.50e+07, 3.00e+07
    chunk 3.00e+07, 3.50e+07
    chunk 3.50e+07, 4.00e+07
    chunk 4.00e+07, 4.50e+07
    chunk 4.50e+07, 5.00e+07
    chunk 5.00e+07, 5.50e+07
    chunk 5.50e+07, 6.00e+07
    chunk 6.00e+07, 6.18e+07
=======INGESTING gene_info TOOK 1.59e+00 minutes=======
=======INGESTING GENE TO ENSEMBL=======
=======INGESTING gene2ensembl TOOK 7.17e-01 minutes=======
=======INGESTING ORTHOLOGS=======
=======DELETING INDEX tmp_gene_to_species_idx======
=======CREATING INDEX tmp_gene_to_species_idx=======
=======DELETING INDEX tmp_gene_to_species_idx======
gene0 15811231
gene1 15811231
    GOT GRAPH
    group 1 decrement 7.170e+02 of 1.593e+07
    group 11 decrement 7.872e+03 of 1.593e+07
    group 111 decrement 7.907e+04 of 1.593e+07


/Users/scott.daniel/KnowledgeEngineering/mmc_gene_mapper/src/mmc_gene_mapper/create_db/ortholog_ingestion.py:383: InvalidOrthologGeneWarning: The following genes had no species assigned to them and were not ingested
[3512700]
  warnings.warn(warning_msg, category=InvalidOrthologGeneWarning)


inserting 15934111 values with citation 0
=======INGESTING gene_orthologs TOOK 2.18e+00 minutes=======
=======INGESTING /Users/scott.daniel/KnowledgeEngineering/mmc_gene_mapper/data/ensembl_download/monodelphis_domestica.jsonld=======
    INGESTING 34985 GENES
=======INGESTING /Users/scott.daniel/KnowledgeEngineering/mmc_gene_mapper/data/ensembl_download/nannospalax_galili.jsonld=======
    INGESTING 24636 GENES
=======INGESTING /Users/scott.daniel/KnowledgeEngineering/mmc_gene_mapper/data/ensembl_download/ictalurus_punctatus.jsonld=======
    INGESTING 39101 GENES
=======INGESTING /Users/scott.daniel/KnowledgeEngineering/mmc_gene_mapper/data/ensembl_download/sorex_araneus.jsonld=======
    INGESTING 19134 GENES
=======INGESTING /Users/scott.daniel/KnowledgeEngineering/mmc_gene_mapper/data/ensembl_download/oncorhynchus_kisutch.jsonld=======
    INGESTING 51004 GENES
=======INGESTING /Users/scott.daniel/KnowledgeEngineering/mmc_gene_mapper/data/ensembl_download/zalophus_californianus.js

In [10]:
gene_mapper.db_path

PosixPath('/Users/scott.daniel/KnowledgeEngineering/mmc_gene_mapper/data/gene_mapper_example.db')

## Contents of the database backing the mapper

The database backing the `gene_mapper` contains the following tables

### Metadata tables

#### authority
The `authority` table just tracks the gene identifying authorities the database knows about (i.e. ENSEMBL and NCBI). it consists of the columns

- id -- an integer used for internal indexing
- name -- the human-readable name of the authority

#### citation
The `citation` table tracks all of the datasets used to justify the various mappings (symbol-to-identifier, ENSEMBL-to-NCBI, orthologs, etc.). It consists of the columns

- id -- an integer used for internal indexing
- name -- a short, human-readable name for the citation (this will be used whenever you have to specify a citation when calling the `gene_mapper`'s methods
- metadata -- a JSON serialized dict containing whatever information is necessary to recreate or specify the data backing this citation. This is a very free-form column as different entries will be specified in different ways (see below)

We can list the available authorities like so

In [11]:
authority_list = gene_mapper.get_all_authorities()
print(authority_list)

['NCBI', 'ENSEMBL']


Similarly, we can list the available citations.

In [12]:
citation_list = gene_mapper.get_all_citations()
print([c['name'] for c in citation_list])

['NCBI', 'ENSEMBL-13616-114', 'ENSEMBL-1026970-114', 'ENSEMBL-7998-114', 'ENSEMBL-42254-114', 'ENSEMBL-8019-114', 'ENSEMBL-9704-114', 'ENSEMBL-425635-114', 'ENSEMBL-106582-114', 'ENSEMBL-9135-114', 'ENSEMBL-9838-114', 'ENSEMBL-38772-114', 'ENSEMBL-10020-114', 'ENSEMBL-9157-114', 'ENSEMBL-48699-114', 'ENSEMBL-161767-114', 'ENSEMBL-173247-114', 'ENSEMBL-9646-114', 'ENSEMBL-48698-114', 'ENSEMBL-60711-114', 'ENSEMBL-29139-114', 'ENSEMBL-1328070-114', 'ENSEMBL-441366-114', 'ENSEMBL-100819-114', 'ENSEMBL-9739-114', 'ENSEMBL-9541-114', 'ENSEMBL-38626-114', 'ENSEMBL-51338-114', 'ENSEMBL-9089-114', 'ENSEMBL-80966-114', 'ENSEMBL-8078-114', 'ENSEMBL-59463-114', 'ENSEMBL-61819-114', 'ENSEMBL-94237-114', 'ENSEMBL-61383-114', 'ENSEMBL-132585-114', 'ENSEMBL-591936-114', 'ENSEMBL-13735-114', 'ENSEMBL-8128-114', 'ENSEMBL-30461-114', 'ENSEMBL-43179-114', 'ENSEMBL-40217-114', 'ENSEMBL-308060-114', 'ENSEMBL-10116-114', 'ENSEMBL-8508-114', 'ENSEMBL-41447-114', 'ENSEMBL-9555-114', 'ENSEMBL-68415-114', 'ENSE

As mentioned above, different citations contain different metadata. For instance, the `NCBI` citation (referring to the data downloaded from the NCBI FTP server) contains a list of the files downloaded their hashes, and the date they were downloaded on (NCBI updates its FTP server daily; it's not clear that they keep readily available version information around, though we can certainly investigate that). This is what the `NCBI` citation metadata looks like:

In [13]:
for citation in citation_list:
    if citation['name'] == 'NCBI':
        print(json.dumps(citation, indent=2))

{
  "name": "NCBI",
  "metadata": {
    "gene_info.gz": {
      "host": "ftp.ncbi.nlm.nih.gov",
      "src_path": "gene/DATA/gene_info.gz",
      "hash": "md5:1465f53bf21ce2ef2857ef24b66fddea",
      "downloaded_on": "2025-07-14-17-19-10"
    },
    "gene2ensembl.gz": {
      "host": "ftp.ncbi.nlm.nih.gov",
      "src_path": "gene/DATA/gene2ensembl.gz",
      "hash": "md5:5437827625a151f72e4aac5c38443c66",
      "downloaded_on": "2025-04-22-00-18-54"
    },
    "gene_orthologs.gz": {
      "host": "ftp.ncbi.nlm.nih.gov",
      "src_path": "gene/DATA/gene_orthologs.gz",
      "hash": "md5:4fedb187701012fb23fd9707b36c9c3e",
      "downloaded_on": "2025-07-14-17-19-34"
    }
  }
}


The citations based on BICAN bkbit files, contain all of the GenomeAnnotation and GenomeAssembly information in those files.

In [14]:
for citation in citation_list:
    if citation['name'] == 'ENSEMBL-10090-98':
        print(json.dumps(citation, indent=2))

{
  "name": "ENSEMBL-10090-98",
  "metadata": {
    "biolink:OrganismTaxon": {
      "id": "NCBITaxon:10090",
      "iri": "http://purl.obolibrary.org/obo/NCBITaxon_10090",
      "category": [
        "biolink:OrganismTaxon"
      ],
      "name": "house mouse",
      "full_name": "Mus musculus"
    },
    "bican:GenomeAssembly": {
      "id": "NCBIAssembly:GCF_000001635.26",
      "category": [
        "bican:GenomeAssembly"
      ],
      "name": "GRCm38",
      "in_taxon": [
        "NCBITaxon:10090"
      ],
      "in_taxon_label": "Mus musculus",
      "version": "26"
    },
    "bican:GenomeAnnotation": {
      "id": "bican:annotation-ENSEMBL-10090-98",
      "category": [
        "bican:GenomeAnnotation"
      ],
      "description": "ENSEMBL Mus musculus Annotation Release 98",
      "in_taxon": [
        "NCBITaxon:10090"
      ],
      "in_taxon_label": "Mus musculus",
      "version": "98",
      "digest": [
        "urn:uuid:98e048e0-0ce1-42a2-90a3-e2bda0c62a5e"
      ],
  

The `HMBA` citation is an ingest of a lookup table created by our science team. As such, it has the least detailed information.

In [15]:
for citation in citation_list:
    if citation['name'] == 'HMBA':
        print(json.dumps(citation, indent=2))

### Data tables

The data tables in the database are

#### NCBI_species
The `NCBI_species` table just ingests the NCBI organism taxonomy and records
- id -- the NCBI taxon ID of the species
- name -- the human readable name of the species. **Note:** we record all possible names ("common", "scientific", etc.) of the species in different rows. That way, users can give us whatever name they have for a species and, if possible, we can return an `id` for cross-referencing with other tables.

This is meant to be hidden from the user. The user specifies the species when performing a mapping and the gene_mapper makes the necessary database call. Below, we will make the call "by hand", so you can see what is in the database.

In [16]:
with sqlite3.connect(gene_mapper.db_path) as conn:
    cursor = conn.cursor()
    for name in ('house mouse', 'Mus musculus', 'mouse', 'human', 'Homo sapiens'):
        result = cursor.execute(
            """
            SELECT id FROM NCBI_species WHERE name=?
            """,
            (name,)
        ).fetchall()
        print(name,result)

house mouse [(10090,)]
Mus musculus [(10090,)]
mouse [(10088,), (10090,)]
human [(9606,)]
Homo sapiens [(9606,)]


**Note:** in the case of "mouse", there is more than one matching taxon. Calling a mapping with `species="mouse"` will fail.

In [17]:
import traceback

mouse_symbols = ["Xkr4", "Npbwr1", "not_a_symbol", "Rrs1"]
try:
    gene_mapper.map_genes(
        gene_list=mouse_symbols,
        dst_species="mouse",
        dst_authority="ENSEMBL"
    )
except Exception as err:
    msg = traceback.format_exc()
    print(
        f"Failed with error message:\n=======\n{msg}"
    )

Failed with error message:
Traceback (most recent call last):
  File "/var/folders/8b/hnw5vq8s20jbpz51wdhd11fr0000gp/T/ipykernel_13286/4200133508.py", line 5, in <module>
    gene_mapper.map_genes(
  File "/Users/scott.daniel/KnowledgeEngineering/mmc_gene_mapper/src/mmc_gene_mapper/mapper/mapper.py", line 184, in map_genes
    dst_species = query_utils.get_species(
                  ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/scott.daniel/KnowledgeEngineering/mmc_gene_mapper/src/mmc_gene_mapper/query_db/query.py", line 35, in get_species
    species_taxon = _get_species_taxon(
                    ^^^^^^^^^^^^^^^^^^^
  File "/Users/scott.daniel/KnowledgeEngineering/mmc_gene_mapper/src/mmc_gene_mapper/query_db/query.py", line 87, in _get_species_taxon
    raise ValueError(
ValueError: 2 species match name mouse
[(10088,), (10090,)]



A function is provided for listing all of the species the database knows about.

**Note:** This is not a list of all of the species for which the database has gene information. It is merely a list of the species that can be associated with a taxon ID in the database (for cross-referencing with the tables that contain gene information).

In [18]:
species_list = gene_mapper.get_all_species()
print(species_list[2000000:2000005])

['Lathraea japonica Miq., 1867', 'Lathraea squamaria', 'Lathraea squamaria L., 1753', 'Lathraeocarpa', 'Lathraeocarpa Bremek.']


In [19]:
'Homo sapiens' in species_list

True

In [20]:
len(species_list)

4070242

#### gene
The `gene` table contains all of the information about individual genes ingested into the database. Columns are
- authority -- an integer for cross-referencing the `authority` table
- citation -- an integer for cross-referencing the `citation` table
- species_taxon -- an integer for cross-refereneing the `NCBI_species` table
- id -- an integer for internal indexing
- identifier -- a string. The full unique identifier (e.g. ENSMUS0000G12345) of the gene
- symbol -- a string. The human-readable name of the gene (**Note:** the BICAN bkbit files contain `symbol` and `name` entries for genes. If these differ, both are ingested as a separate row in the `gene` table with `name` being recorded in `symbol` as appropriate).

#### gene_equivalence
The `gene_equivalence` table records mappings between authorities (ENSEMBL and NCBI). Its columns are
- species_taxon -- an integer for cross-referencing with the `NCBI_species` table (in what species are we looking for equivalences?)
- citation -- an integer for cross-referencing with the `citation` table (who said these genes were equivalent?)
- authority0 -- an integer for cross-referencing with the `authority` table
- gene0 -- an integer; the ID of the gene in `authority0`
- authority1 -- an integer for cross-referncing with the `authority` table
- gene1 -- an integer; the ID of the gene in `authority1` that is equivalent to `gene0` according to `citation`.

**Note:** we record equivalences symmetrically, so that every `(gene0, gene1)` pair is also recorded as `(gene1, gene0)` so that users do not have to worry about which index (`gene0` or `gene1`) their input data is compared to.

#### gene_ortholog
The `gene_ortholog` table records cross-species ortholog relationships. Its columns are
- authority -- an integer for cross-referencing the `authority` table (are we looking for NCBI orthologs or ENSEMBL orthologs?)
- citation -- an integer for cross-referencing the `citation` table (who said these genes were orthologs?)
- species -- an integer for cross-referencing the `NCBI_species` table
- gene -- an integer, the ID of the gene in `species` whose orthologs we are looking for
- ortholog_group -- an integer; genes with the same value of `ortholog_group` are orthologs of each other


## Actually mapping data

The `gene_mapper` provides a single function to map gene identifiers from one form `(authority, species)` to another. To run it you specify
- the list of gene identifeirs you have
- the name of the species you want your genes mapped into (the mapper will automatically detect the species your genes are already in(
- the authority of gene (ENSEMBL or NCBI) you want your genes mapped into
- (optionally) the name of the citation to use for any cross species ortholog mappings (default is to use the NCBI lookup table)

The mapper will then go through the necessary transformations and return a mapped list of gene identifiers for you.

### Mapping symbols to identifiers



In [21]:
mouse_symbols = ["Xkr4", "Npbwr1", "not_a_symbol", "Rrs1"]
mouse_ens_mapping = gene_mapper.map_genes(
    gene_list=mouse_symbols,
    dst_species="Mus musculus",
    dst_authority="ENSEMBL"
)
print(json.dumps(mouse_ens_mapping, indent=2))

{
  "metadata": [
    {
      "mapping": {
        "axis": "authority",
        "from": "symbol",
        "to": "ENSEMBL"
      },
      "citation": {
        "name": "ENSEMBL-10090-98",
        "metadata": {
          "biolink:OrganismTaxon": {
            "id": "NCBITaxon:10090",
            "iri": "http://purl.obolibrary.org/obo/NCBITaxon_10090",
            "category": [
              "biolink:OrganismTaxon"
            ],
            "name": "house mouse",
            "full_name": "Mus musculus"
          },
          "bican:GenomeAssembly": {
            "id": "NCBIAssembly:GCF_000001635.26",
            "category": [
              "bican:GenomeAssembly"
            ],
            "name": "GRCm38",
            "in_taxon": [
              "NCBITaxon:10090"
            ],
            "in_taxon_label": "Mus musculus",
            "version": "26"
          },
          "bican:GenomeAnnotation": {
            "id": "bican:annotation-ENSEMBL-10090-98",
            "category": [
       

For ease of reading, here is the output broken down into metadata

In [22]:
print(json.dumps(mouse_ens_mapping['metadata'], indent=2))

[
  {
    "mapping": {
      "axis": "authority",
      "from": "symbol",
      "to": "ENSEMBL"
    },
    "citation": {
      "name": "ENSEMBL-10090-98",
      "metadata": {
        "biolink:OrganismTaxon": {
          "id": "NCBITaxon:10090",
          "iri": "http://purl.obolibrary.org/obo/NCBITaxon_10090",
          "category": [
            "biolink:OrganismTaxon"
          ],
          "name": "house mouse",
          "full_name": "Mus musculus"
        },
        "bican:GenomeAssembly": {
          "id": "NCBIAssembly:GCF_000001635.26",
          "category": [
            "bican:GenomeAssembly"
          ],
          "name": "GRCm38",
          "in_taxon": [
            "NCBITaxon:10090"
          ],
          "in_taxon_label": "Mus musculus",
          "version": "26"
        },
        "bican:GenomeAnnotation": {
          "id": "bican:annotation-ENSEMBL-10090-98",
          "category": [
            "bican:GenomeAnnotation"
          ],
          "description": "ENSEMBL Mus m

and actual mapping

In [23]:
print(json.dumps(mouse_ens_mapping['gene_list'], indent=2))

[
  "ENSMUSG00000051951",
  "ENSMUSG00000033774",
  "symbol:ENSEMBL:UNMAPPABLE_NO_MATCH_0",
  "ENSMUSG00000061024"
]


Here is a similar call mapping symbols to identifiers in NCBI

In [24]:
mouse_ncbi_mapping = gene_mapper.map_genes(
    gene_list=mouse_symbols,
    dst_species="Mus musculus",
    dst_authority="NCBI"
)
print(json.dumps(mouse_ncbi_mapping, indent=2))

{
  "metadata": [
    {
      "mapping": {
        "axis": "authority",
        "from": "symbol",
        "to": "NCBI"
      },
      "citation": {
        "name": "NCBI",
        "metadata": {
          "gene_info.gz": {
            "host": "ftp.ncbi.nlm.nih.gov",
            "src_path": "gene/DATA/gene_info.gz",
            "hash": "md5:1465f53bf21ce2ef2857ef24b66fddea",
            "downloaded_on": "2025-07-14-17-19-10"
          },
          "gene2ensembl.gz": {
            "host": "ftp.ncbi.nlm.nih.gov",
            "src_path": "gene/DATA/gene2ensembl.gz",
            "hash": "md5:5437827625a151f72e4aac5c38443c66",
            "downloaded_on": "2025-04-22-00-18-54"
          },
          "gene_orthologs.gz": {
            "host": "ftp.ncbi.nlm.nih.gov",
            "src_path": "gene/DATA/gene_orthologs.gz",
            "hash": "md5:4fedb187701012fb23fd9707b36c9c3e",
            "downloaded_on": "2025-07-14-17-19-34"
          }
        }
      }
    }
  ],
  "gene_list": [
    "NC

### Mapping from ENSEMBL to NCBI


In [25]:
ens_to_ncbi = gene_mapper.map_genes(
    dst_authority='NCBI',
    gene_list=["ENSMUSG00000051951",
               "ENSMUSG00000030337",
               "ENSMUSG00000087247",
               "nope",
               "ENSMUSG00000025911"],
    dst_species="Mus musculus"
)
print(json.dumps(ens_to_ncbi, indent=2))

{
  "metadata": [
    {
      "mapping": {
        "axis": "authority",
        "from": "symbol",
        "to": "NCBI"
      },
      "citation": {
        "name": "NCBI",
        "metadata": {
          "gene_info.gz": {
            "host": "ftp.ncbi.nlm.nih.gov",
            "src_path": "gene/DATA/gene_info.gz",
            "hash": "md5:1465f53bf21ce2ef2857ef24b66fddea",
            "downloaded_on": "2025-07-14-17-19-10"
          },
          "gene2ensembl.gz": {
            "host": "ftp.ncbi.nlm.nih.gov",
            "src_path": "gene/DATA/gene2ensembl.gz",
            "hash": "md5:5437827625a151f72e4aac5c38443c66",
            "downloaded_on": "2025-04-22-00-18-54"
          },
          "gene_orthologs.gz": {
            "host": "ftp.ncbi.nlm.nih.gov",
            "src_path": "gene/DATA/gene_orthologs.gz",
            "hash": "md5:4fedb187701012fb23fd9707b36c9c3e",
            "downloaded_on": "2025-07-14-17-19-34"
          }
        }
      }
    },
    {
      "mapping": {
   

Here is a call mapping from NCBI to ENSEMBL

In [26]:
ncbi_to_ens = gene_mapper.map_genes(
    dst_authority='ENSEMBL',
    gene_list=["NCBIGene:67269", "NCBIGene:54200", "nope", "NCBIGene:70911"],
    dst_species="Mus musculus"
)
print(json.dumps(ncbi_to_ens, indent=2))

{
  "metadata": [
    {
      "mapping": {
        "axis": "authority",
        "from": "symbol",
        "to": "ENSEMBL"
      },
      "citation": {
        "name": "ENSEMBL-10090-98",
        "metadata": {
          "biolink:OrganismTaxon": {
            "id": "NCBITaxon:10090",
            "iri": "http://purl.obolibrary.org/obo/NCBITaxon_10090",
            "category": [
              "biolink:OrganismTaxon"
            ],
            "name": "house mouse",
            "full_name": "Mus musculus"
          },
          "bican:GenomeAssembly": {
            "id": "NCBIAssembly:GCF_000001635.26",
            "category": [
              "bican:GenomeAssembly"
            ],
            "name": "GRCm38",
            "in_taxon": [
              "NCBITaxon:10090"
            ],
            "in_taxon_label": "Mus musculus",
            "version": "26"
          },
          "bican:GenomeAnnotation": {
            "id": "bican:annotation-ENSEMBL-10090-98",
            "category": [
       

To simulate "production scale," we will now download all of the genes in the Yao et al. 2023 10X gene panel and try mappping them from ENSEMBL to NCBI to see how long it takes to process that many genes.

In [27]:
wmb_gene_df = abc_cache.get_metadata_dataframe(
    directory='WMB-10X',
    file_name='gene'
)
wmb_genes = wmb_gene_df.gene_identifier.values
t0 = time.time()
wmb_ncbi = gene_mapper.map_genes(
    dst_authority='NCBI',
    gene_list=wmb_genes,
    dst_species='Mus musculus',
)
dur = time.time()-t0
print(f'mapping {len(wmb_genes)} genes took {dur:.2e} seconds')

n_mapped = 0
for k in wmb_ncbi['gene_list']:
    if 'UNMAPPABLE' not in k:
        n_mapped += 1
print(f'{n_mapped} genes had unique NCBI matches')


mapping 32285 genes took 1.68e+01 seconds
24503 genes had unique NCBI matches


Now we will download the marker genes used by MapMyCells for the Whole Mouse Brain taxonomy and try mapping them from ENSEMBL to NCBI.

In [28]:
src_path = "https://allen-brain-cell-atlas.s3-us-west-2.amazonaws.com/mapmycells/WMB-10X/20240831/mouse_markers_230821.json"
wmb_marker_path = pathlib.Path('../data/mouse_markers_230821.json')
if not wmb_marker_path.is_file():
    print(f"downloading {wmb_marker_path}")
    process = subprocess.Popen(
        args=[
           "wget",
            src_path,
            "-O",
            str(wmb_marker_path),
        ],
        stderr=subprocess.DEVNULL
    )
    process.wait()

In [29]:
marker_genes = set()
with open(wmb_marker_path, 'rb') as src:
    marker_lookup = json.load(src)
for key in marker_lookup:
    if key in ('log', 'metadata'):
        continue
    marker_genes = marker_genes.union(set(marker_lookup[key]))
marker_genes = sorted(marker_genes)

Let's now map our mouse marker genes from ENSEMBL to NCBI

In [30]:
print(f'{len(marker_genes)} marker genes')
t0 = time.time()
marker_ncbi = gene_mapper.map_genes(
    dst_authority='NCBI',
    gene_list=marker_genes,
    dst_species='Mus musculus'
)
dur = time.time()-t0
print(f'mappiing {len(marker_genes)} genes took {dur:.2e} seconds')

n_mapped = 0
n_one_to_one = 0
for k in marker_ncbi['gene_list']:
    if 'UNMAPPABLE' not in k:
        n_mapped += 1

print(f'{n_mapped} genes had unique NCBI matches')


6558 marker genes
mappiing 6558 genes took 5.46e+00 seconds
6493 genes had unique NCBI matches


### Ortholog mapping

To test ortholog mapping, let's take the genes from Yao et al. 2023 that only mapped to one NCBI gene and map them to human orthologs.

First let's just map 5 genes so that we can display the output and see what it looks like

In [31]:
test_orthologs = gene_mapper.map_genes(
    dst_authority='ENSEMBL',
    dst_species='human',
    gene_list=list(wmb_ncbi['gene_list'][10:15]) + ['not_an_actual_gene']
)
print(json.dumps(test_orthologs['gene_list'], indent=2))

[
  "ENSG00000187735",
  "ENSG00000147509",
  "ortholog:UNMAPPABLE_NO_MATCH_0",
  "ENSG00000047249",
  "ENSG00000082556",
  "symbol:NCBI:UNMAPPABLE_NO_MATCH_0"
]


Now let's map all 24,000 genes and see how long that takes

In [32]:
t0 = time.time()
wmb_to_whb_orthologs = gene_mapper.map_genes(
    dst_authority='ENSEMBL',
    dst_species='human',
    gene_list=wmb_ncbi['gene_list']
)
dur = time.time()-t0
print(f'mapping {len(wmb_ncbi['gene_list'])} orthologs took {dur:.2e} seconds')
n_mapped = 0
for k in wmb_to_whb_orthologs['gene_list']:
    if 'UNMAPPABLE' not in k:
        n_mapped += 1
print(f'{n_mapped} genes had orthologs')


mapping 32285 orthologs took 2.45e+01 seconds
16459 genes had orthologs


In [33]:
print(json.dumps(wmb_to_whb_orthologs['metadata'], indent=2))

[
  {
    "mapping": {
      "axis": "authority",
      "from": "symbol",
      "to": "NCBI"
    },
    "citation": {
      "name": "NCBI",
      "metadata": {
        "gene_info.gz": {
          "host": "ftp.ncbi.nlm.nih.gov",
          "src_path": "gene/DATA/gene_info.gz",
          "hash": "md5:1465f53bf21ce2ef2857ef24b66fddea",
          "downloaded_on": "2025-07-14-17-19-10"
        },
        "gene2ensembl.gz": {
          "host": "ftp.ncbi.nlm.nih.gov",
          "src_path": "gene/DATA/gene2ensembl.gz",
          "hash": "md5:5437827625a151f72e4aac5c38443c66",
          "downloaded_on": "2025-04-22-00-18-54"
        },
        "gene_orthologs.gz": {
          "host": "ftp.ncbi.nlm.nih.gov",
          "src_path": "gene/DATA/gene_orthologs.gz",
          "hash": "md5:4fedb187701012fb23fd9707b36c9c3e",
          "downloaded_on": "2025-07-14-17-19-34"
        }
      }
    }
  },
  {
    "mapping": {
      "axis": "species",
      "from": {
        "name": "Balb/c mouse",
        "t

To map orthologs using the "by-hand" look up table created by our science team, change the kwarg `citation='HMBA'`

In [35]:
#t0 = time.time()
#wmb_to_whb_orthologs_hmba = gene_mapper.map_genes(
#    dst_authority='NCBI',
#    dst_species='human',
#    gene_list=wmb_ncbi['gene_list'],
#    ortholog_citation='HMBA'
#)
#dur = time.time()-t0
#print(f'mapping {len(wmb_ncbi["gene_list"])} orthologs took {dur:.2e} seconds')
#n_mapped = 0
#for k in wmb_to_whb_orthologs_hmba['gene_list']:
#    if 'UNMAPPABLE' not in k:
#            n_mapped += 1
#print(f'{n_mapped} genes had orthologs')


#### Map from mouse to naked mole rat

Now, inspired by a recent community forum post, let's take our mouse marker genes, convert them into NCBI identifiers, and finally convert those in to naked mole rat orthologs.

In [36]:
t0 = time.time()

naked_mole_rat = gene_mapper.map_genes(
    dst_authority='ENSEMBL',
    dst_species='naked mole rat',
    gene_list=marker_genes,
    ortholog_citation='NCBI'
)

dur = time.time()-t0
print(f"mapping took {dur:.2e} seconds")
n_mapped = 0
for g in naked_mole_rat['gene_list']:
    if 'UNMAPPABLE' not in g:
        n_mapped += 1
print(f"{n_mapped} genes of {len(marker_genes)} had orthologs")
print(marker_genes[:5])
print(naked_mole_rat['gene_list'][:5])

mapping took 6.75e+00 seconds
5048 genes of 6558 had orthologs
['ENSMUSG00000000028', 'ENSMUSG00000000037', 'ENSMUSG00000000056', 'ENSMUSG00000000058', 'ENSMUSG00000000078']
['ENSHGLG00000015159', 'NCBI:ENSEMBL:UNMAPPABLE_NO_MATCH_0', 'ENSHGLG00000008789', 'ENSHGLG00000002962', 'ENSHGLG00000007077']


Here is the metadata for the naked mole rat ortholog mapping

In [37]:
print(json.dumps(naked_mole_rat['metadata'], indent=2))

[
  {
    "mapping": {
      "axis": "authority",
      "from": "ENSEMBL",
      "to": "NCBI"
    },
    "citation": {
      "name": "NCBI",
      "metadata": {
        "gene_info.gz": {
          "host": "ftp.ncbi.nlm.nih.gov",
          "src_path": "gene/DATA/gene_info.gz",
          "hash": "md5:1465f53bf21ce2ef2857ef24b66fddea",
          "downloaded_on": "2025-07-14-17-19-10"
        },
        "gene2ensembl.gz": {
          "host": "ftp.ncbi.nlm.nih.gov",
          "src_path": "gene/DATA/gene2ensembl.gz",
          "hash": "md5:5437827625a151f72e4aac5c38443c66",
          "downloaded_on": "2025-04-22-00-18-54"
        },
        "gene_orthologs.gz": {
          "host": "ftp.ncbi.nlm.nih.gov",
          "src_path": "gene/DATA/gene_orthologs.gz",
          "hash": "md5:4fedb187701012fb23fd9707b36c9c3e",
          "downloaded_on": "2025-07-14-17-19-34"
        }
      }
    }
  },
  {
    "mapping": {
      "axis": "species",
      "from": {
        "name": "Balb/c mouse",
        "